In [1]:
# Add to environment variables PYTHONPATH = C:\Users\JFC_WIN\Documents\OpenSimula\src
import OpenSimula as osm
import pandas as pd

bestest_building_600 = {
    "name": "Bestest Building",
    "time_step": 3600,
    "n_time_steps": 8760,
    "initial_time": "01/01/2001 00:00:00",
    "components": [
        # MET_FILE
        {
            "type": "File_met",
            "name": "sevilla",
            "file_name": "met_files/sevilla.met"
        },
        # CONSTRUCTION
        {
            "type": "Material",
            "name": "Enlucido",
            "conductivity": 0.16,
            "density": 950,
            "specific_heat": 840
        },
        {
            "type": "Material",
            "name": "Aislamiento",
            "conductivity": 0.04,
            "density": 12,
            "specific_heat": 840
        },
        {
            "type": "Material",
            "name": "Madera pared",
            "conductivity": 0.14,
            "density": 530,
            "specific_heat": 900
        },
        {
            "type": "Material",
            "name": "Aislamiento suelo",
            "conductivity": 0.04,
            "density": .1,
            "specific_heat": .1,
        },
        {
            "type": "Material",
            "name": "Madera suelo",
            "conductivity": 0.14,
            "density": 650,
            "specific_heat": 1200,
        },
        {
            "type": "Material",
            "name": "Lecho_rocas",
            "conductivity": 0.14,
            "density": 650,
            "specific_heat": 1200,
        },
        {
            "type": "Construction",
            "name": "Pared 600",
            "solar_absortivity": [0.6, 0.6],
            "materials": ["Enlucido", "Aislamiento", "Madera pared"],
            "thicknesses": [0.12, 0.066, 0.009],
        },
        {
            "type": "Construction",
            "name": "Suelo 600",
            "solar_absortivity": [0.6, 0.6],
            "materials": ["Aislamiento suelo", "Madera suelo"],
            "thicknesses": [1.003, 0.025],
        },
        {
            "type": "Construction",
            "name": "Techo 600",
            "solar_absortivity": [0.6, 0.6],
            "materials": ["Enlucido", "Aislamiento", "Madera pared"],
            "thicknesses": [0.010, 0.01118, 0.019],
        },
        {
            "type": "Window",
            "name": "Vidrio doble",
            "solar_absortivity": [0.13, 0.13],
            "solar_transmisivity": [0.66, 0.66],
            "R_glazing": 0.921,
            "R_frame": 0.921,
            "frame_fraction": 0
        },
        # BUILDING
        {
            "type": "Building",
            "name": "Bestest 600",
            "file_met": "sevilla"
        },
        {
            "type": "Space_type",
            "name": "bestest space",
            "aux_variables": [],
            "people_density": "0",
            "light_density": "0",
            "other_gains_density": "4.1667",
            "other_gains_radiant_fraction": 0.6,
            "infiltration": "0.5"
        },
        {
            "type": "Space",
            "name": "space_1",
            "building": "Bestest 600",
            "space_type": "bestest space",
            "floor_area": 48,
            "volume": 48*2.7,
        },
        # Surfaces
        {
            "type": "Exterior_surface",
            "name": "north_wall",
            "virtual": False,
            "construction": "Pared 600",
            "space": "space_1",
            "area": 8*2.7,
            "azimuth": 180,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "east_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "area": 6*2.7,
            "azimuth": 90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "south_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "area": 8*2.7,
            "azimuth": 0,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "west_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "area": 6*2.7,
            "azimuth": -90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "roof",
            "construction": "Techo 600",
            "space": "space_1",
            "area": 8*6,
            "azimuth": 0,
            "altitude": 90
        },
        {
            "type": "Underground_surface",
            "name": "floor",
            "construction": "Suelo 600",
            "space": "space_1",
            "area": 8*6,
            "azimuth": 0,
            "altitude": -90
        },
    ],
}

In [2]:
sim = osm.Simulation()
pro = osm.Project("pro", sim)
pro.read_dict(bestest_building_600)
pro.simulate()

Reading project data from dictonary
Reading completed.
Checking project: Bestest Building
ok
Simulating Bestest Building: 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  End


/Users/jfc/Documents/OpenSimula/src/OpenSimula/components/Exterior_surface.py:94: RuntimeWarning: overflow encountered in scalar multiply
  self.f_0 = self.net_area * (- p_0 - self.parameter("h_cv").value[0] * self._T_ext - h_rd * T_rm - self.radiant_property(
/Users/jfc/Documents/OpenSimula/src/OpenSimula/components/Building.py:305: RuntimeWarning: invalid value encountered in subtract
  self.TS_vector = np.matmul(self.KS_inv_matrix, self.FS_vector) - np.matmul(


In [13]:
pro.component("Pared 600")._coef_Q

array([ 1.00000000e+00, -1.21231753e+00,  3.23088313e-01, -1.26285210e-02,
        2.80344262e-05, -1.26091417e-09,  4.31507581e-16])

In [8]:
df_met = pro.component("sevilla").variable_dataframe()
df_space = pro.component("space_1").variable_dataframe()
df_wall = pro.component("south_wall").variable_dataframe()

df_graph = pd.DataFrame()
df_graph["date"] = df_met["date"]
df_graph["T_ext"] = df_met["temperature [°C]"]
df_graph["T_z"] = df_space["temperature [°C]"]
df_graph["T_s0"] = df_wall["T_s0 [°C]"]
df_graph["T_s1"] = df_wall["T_s1 [°C]"]

df_wall.head(24)

,date,T_s0 [°C],T_s1 [°C],T_rm [°C],E_dir0 [W/m²],E_dif0 [W/m²],q_cd0 [W/m²],q_cd1 [W/m²],p_0 [W/m²],p_1 [W/m²]
0,2001-01-01 00:00:00,-6.141498e-01,1.856677e+01,-0.987367,0.000000,0.000000,-1.385256e+02,-2.518705e+00,-1.343962e+02,-3.518109e+01
1,2001-01-01 01:00:00,2.271403e+01,1.689317e+01,-1.060133,0.000000,0.000000,4.352738e+02,-3.484476e+00,2.826391e+02,-3.319974e+01
2,2001-01-01 02:00:00,-2.993484e+01,1.632932e+01,9.373309,0.000000,0.000000,-1.070485e+03,-1.864710e+00,-8.693226e+02,-3.059474e+01
3,2001-01-01 03:00:00,1.282554e+02,1.595430e+01,8.873309,0.000000,0.000000,2.807031e+03,-1.149004e+00,1.945165e+03,-2.919927e+01
4,2001-01-01 04:00:00,-2.935139e+02,1.556320e+01,8.367971,0.000000,0.000000,-7.486344e+03,-1.106743e+00,-5.513946e+03,-2.852241e+01
5,2001-01-01 05:00:00,8.285451e+02,1.500538e+01,7.773309,0.000000,0.000000,1.994514e+04,-9.481136e-01,1.437737e+04,-2.724035e+01
6,2001-01-01 06:00:00,-2.163404e+03,1.484668e+01,7.273309,0.000000,0.000000,-5.314943e+04,-1.352657e+00,-3.861149e+04,-2.774467e+01
7,2001-01-01 07:00:00,5.807345e+03,1.363160e+01,6.773309,0.000000,0.000000,1.416241e+05,-3.245550e-01,1.025991e+05,-2.356946e+01
8,2001-01-01 08:00:00,-1.543394e+04,1.524447e+01,6.316014,0.000000,0.416377,-3.773851e+05,-3.118963e+00,-2.736700e+05,-3.189159e+01
9,2001-01-01 09:00:00,4.116717e+04,9.453111e+00,6.664058,1.307309,9.937934,1.005620e+06,4.499059e+00,7.289788e+05,-6.916478e+00


In [5]:
import plotly.express as px

fig = px.line(df_graph,x='date',y=['T_ext','T_z','T_s0','T_s1'],height=400)
fig.show()